 PPからlocationシーケンスの生成

In [1]:
import pandas as pd 
import numpy as np 
import os 
import sys

In [5]:
# trip dataの出発時刻に併せて loc_dataからlon, latを取り出す
base_path0 = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog'#/04_202212old'
dataset_list = ['04_202212old', '04_202301new', '05_202311', 'toyosu_2019/201907-202002', '99_202110/logs']
nameid_list = ['01', '02', '03', 'toyosu', 'shibu21']

for datasetid, dataset in enumerate(dataset_list):
    base_path = os.path.join(base_path0, dataset)
    folderlist = os.listdir(base_path)
    user_log_dict = {}
    user_count_dict = {} # 何日登場したかのカウント
    user_lon_traj = {} # key: userid, value: time trajectory
    user_lat_traj = {} # key: userid, value: act trajectory
    nokitaku_count = 0
    all_count = 0
    folderlist = sorted(folderlist)
    print(folderlist)

    df_latlon = pd.DataFrame(columns=['ユーザーID', '記録日時', '緯度', '経度'])
    for folder in folderlist: # 個人
        if not os.path.isdir(os.path.join(base_path, folder)):
            continue

        filelist = os.listdir(os.path.join(base_path, folder))
        df_trip = pd.read_csv(os.path.join(base_path, folder, 't_trip.csv'), encoding='shift-jis')
        df_loc = pd.read_csv(os.path.join(base_path, folder, 't_loc_data.csv'), encoding='shift-jis')
        df_loc['記録日時'] = pd.to_datetime(df_loc['記録日時'].str.split('.').str[0], errors='coerce') # 秒の小数点以下を切り捨て

        grouped = df_trip.groupby('ユーザーID')
        for user_id, group in grouped: # 個人ごと
            userid = group['ユーザーID'].values[0]
            group = group.sort_values(by='出発時刻')
            acts = list(group['目的コード（active）']) # 系列
            times = list(group['出発時刻']) # 系列

            print(f"User ID: {userid}, Activities: {acts}, Times: {times}")

            for i, time in enumerate(times):
                # lat = df_loc[(df_loc['ユーザーID'] == userid) & (df_loc['記録日時'] == time)]['緯度'].values
                # lon = df_loc[(df_loc['ユーザーID'] == userid) & (df_loc['記録日時'] == time)]['経度'].values
                time = pd.to_datetime(time, errors='coerce')  # ← ここで変換

                user_loc_df = df_loc[df_loc['ユーザーID'] == userid].copy()

                if user_loc_df.empty or pd.isna(time):
                    continue
                # 時刻差を計算
                user_loc_df['time_diff'] = (user_loc_df['記録日時'] - time).abs()

                # 最小差分の行を取得（NaTは自動除外される）
                nearest_row = user_loc_df.loc[user_loc_df['time_diff'].idxmin()]
                lat = nearest_row['緯度']
                lon = nearest_row['経度']

                ## ここでメッシュデータに変換したほうがいい：
                ## 250mメッシュとの対応をとる

                print(f"User ID: {userid}, Time: {time}, Latitude: {lat}, Longitude: {lon}")

                new_row = pd.DataFrame({
                    'ユーザーID': [userid],
                    '記録日時': [time],
                    '緯度': [lat],
                    '経度': [lon]
                })

                df_latlon = pd.concat([df_latlon, new_row], ignore_index=True)
            print(df_latlon)
            sys.exit()
    print(df_latlon)
    sys.exit()
    # /Users/matsunagatakahiro/Desktop/res2025/ActFormer/RoutesFormer/output
    df_latlon.to_csv(('/Users/matsunagatakahiro/Desktop/res2025/ActFormer/RoutesFormer/output' + folder +  'latlon01.csv'), index=False, encoding='shift-jis')

['.DS_Store', '10028_20221212', '10028_20221213', '10028_20221214', '10028_20221215', '10028_20221216', '10028_20221217', '10028_20221218', '10028_20221219', '10028_20221220', '10028_20221221', '10028_20221222', '10028_20221223', '10028_20221224', '10028_20221225', '10028_20230107', '10028_20230108', '10028_20230109', '10028_20230110', '10028_20230111', '10028_20230112', '10028_20230113', '10028_20230124', '10028_20230125', '10028_20230126', '10028_20230127', '10028_20230128', '10028_20230129', '10028_20230130']
User ID: 23001, Activities: [310, 300, 110], Times: ['2022-12-12 07:03:35', '2022-12-12 10:55:36', '2022-12-12 12:25:23']
User ID: 23001, Time: 2022-12-12 07:03:35, Latitude: 35.736969, Longitude: 139.658455
User ID: 23001, Time: 2022-12-12 10:55:36, Latitude: 35.643549, Longitude: 139.670226
User ID: 23001, Time: 2022-12-12 12:25:23, Latitude: 35.738042, Longitude: 139.654243
  ユーザーID                記録日時         緯度          経度
0  23001 2022-12-12 07:03:35  35.736969  139.65845

/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_3993/4205298441.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_latlon = pd.concat([df_latlon, new_row], ignore_index=True)


SystemExit: 

/Users/matsunagatakahiro/Desktop/res2025/myenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
